<a href="https://colab.research.google.com/github/VyzhD/Neural_test/blob/main/test_neural_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import functools
import pandas as pd
import numpy as np
import tensorflow as tf
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
TRAIN_DATA_URL = "/content/drive/MyDrive/train.csv"
TEST_DATA_URL = "/content/drive/MyDrive/eval.csv"

In [11]:
train_file_path = tf.keras.utils.get_file("/content/drive/MyDrive/train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("/content/drive/MyDrive/eval.csv", TEST_DATA_URL)

In [12]:
np.set_printoptions(precision=3, suppress=True)

In [13]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [9]:
Label_column = 'survived'
labels = [0, 1]
print(labels)

[0, 1]


In [14]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Значение искусственно занижено для удобства восприятия.
      label_name=Label_column,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [16]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [17]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'female' b'male']
age                 : [28. 25. 35. 38.  4.]
n_siblings_spouses  : [0 0 0 1 4]
parch               : [0 0 0 0 1]
fare                : [  0.     13.    135.633  71.283  29.125]
class               : [b'First' b'Second' b'First' b'First' b'Third']
deck                : [b'B' b'unknown' b'C' b'C' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Queenstown']
alone               : [b'y' b'y' b'y' b'n' b'n']


In [18]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [19]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [20]:
show_batch(packed_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'male']
class               : [b'Third' b'Third' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']
numeric             : [[28.     0.     0.     7.75 ]
 [11.     4.     2.    31.275]
 [32.     0.     0.    10.5  ]
 [35.     0.     0.     7.125]
 [30.     0.     0.    13.   ]]


In [21]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [23]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
print(desc)

              age  n_siblings_spouses       parch        fare
count  627.000000          627.000000  627.000000  627.000000
mean    29.631308            0.545455    0.379585   34.385399
std     12.511818            1.151090    0.792999   54.597730
min      0.750000            0.000000    0.000000    0.000000
25%     23.000000            0.000000    0.000000    7.895800
50%     28.000000            0.000000    0.000000   15.045800
75%     35.000000            1.000000    0.000000   31.387500
max     80.000000            8.000000    5.000000  512.329200


In [24]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Центрируем данные
  return (data-mean)/std

In [25]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f69cea43598>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [26]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [27]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [28]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [29]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [31]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.349 -0.474
 -0.479 -0.482  1.     0.   ]


In [32]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [33]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [34]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 0s 2ms/step - loss: 0.5074 - accuracy: 0.7624
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4164 - accuracy: 0.8325
Epoch 3/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3991 - accuracy: 0.8389
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3853 - accuracy: 0.8373
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3740 - accuracy: 0.8437
Epoch 6/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3667 - accuracy: 0.8325
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3635 - accuracy: 0.8405
Epoch 8/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3529 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3511 - accurac

In [35]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 1ms/step - loss: 0.4655 - accuracy: 0.8409


Test Loss 0.46545612812042236, Test Accuracy 0.8409090638160706


In [36]:
predictions = model.predict(test_data)

Consider rewriting this model with the Functional API.


In [37]:
# Покажем некоторые результаты
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 99.89%  | Actual outcome:  SURVIVED
Predicted survival: 90.69%  | Actual outcome:  DIED
Predicted survival: 4.04%  | Actual outcome:  DIED
Predicted survival: 5.64%  | Actual outcome:  SURVIVED
Predicted survival: 99.88%  | Actual outcome:  SURVIVED
